In [ ]:
from __init__ import *

# Plotting Functions


In [7]:
def plot_tpr_per_attack(args, results_df):
    """
    Plot the TPR for each attack and watermarking method, with different models as lines.
    The plot will have one row per attack, and one column per watermarking method.
    with the X-axis showing the attack strength and the Y-axis showing the TPR@FPR=0.01.
    """
    results_df['set_fpr'].unique()[0] # set_fpr should be the same for all experiments, so we can just take the first value

    # drop the no_attack case from the results_df
    results_df = results_df[results_df['attack_name'] != 'no_attack']


    attack_names = results_df['attack_name'].unique()
    wm_methods = results_df['wm_method'].unique()
    models = results_df['model_id'].unique()

    # order the attacks and methods based on the order in name_mapping
    attack_names = np.array(sorted(attack_names, key=lambda x: list(ATTACK_NAME_MAPPING.keys()).index(x)))
    wm_methods = np.array(sorted(wm_methods, key=lambda x: list(METHODS_NAME_MAPPING.keys()).index(x)))
    models = np.array(sorted(models, key=lambda x: list(MODEL_NAME_MAPPING.keys()).index(x)))

    # for each attack (rows), plot all 4 WM methods in 4 sublpots (cols), all 2 models as lines

    ncols = wm_methods.shape[0] + 1 # per method, ülus one for title
    nrows = attack_names.shape[0] # for each attack
    fs = 12
    fs_xticks = 8
    fs_yticks = 8
    fs_title = 14
    y_adj = 1.15
    title_height_ratio = 0.15#0.65
    height_correction = 0
    sp_width = 2
    sp_height = 1.25
    # title = ( 
    #     f'Performance of watermarking methods under different attacks\n'
    #     f'for dataset "{args.prompt_dataset}\n"'# for experiments in \n'
    #     f'{args.dataset_identifier}'
    # )

    fig, gs, title_axes = setup_gridspec_figure(
        nrows=nrows, ncols=ncols ,
        fs=fs, fs_title=fs_title,
        y_adj=y_adj, title_height_ratio=title_height_ratio,
        sp_width=sp_width, sp_height=sp_height, height_correction=height_correction,
        wspace=0.1, hspace=0.1,
    )

    # # set the titles for each row, as the attack names
    # for i, ax in enumerate(title_axes):
    #     ax.text(0.5, 0.4, ATTACK_NAME_MAPPING[attack_names[i]]['name'], fontsize=fs_title, fontweight="bold", ha="center", va="center")
                      
    handles, labels = [], []

    # loop through all attacks (rows), and then per attack, loop through all WM methods
    for i, attack_name in enumerate(attack_names): # rows
        attack_df = results_df[results_df['attack_name'] == attack_name]
        if attack_name not in ATTACK_NAME_MAPPING:
            continue

        axes = [fig.add_subplot(gs[2*i +1, j]) for j in range(ncols)]
        for j, wm_method in enumerate(np.concatenate((wm_methods, ["title"]))): # columns
            if wm_method == "title": # last column is title of the attack
                axes[j].axis('off')
                axes[j].text(-0.0, 0.5, ATTACK_NAME_MAPPING[attack_name]['name'], fontsize=fs, ha="left", va="center", rotation=90)
                # if i == 0:
                #     axes[j].set_title('Attacktype', fontsize=fs)
            else:
                wm_df = attack_df[attack_df['wm_method'] == wm_method]
                
                # Set axis direction based on attack type
                if ATTACK_NAME_MAPPING[attack_name]['order'] == 'low-to-high':
                    axes[j].invert_xaxis()
                    
                if i == 0:
                    axes[j].set_title(METHODS_NAME_MAPPING[wm_method], fontsize=fs)
                
                axes[j].set_yticks(np.arange(0, 1.1, 0.5))
                axes[j].set_yticklabels(np.arange(0, 1.1, 0.5), fontsize=fs_yticks)
                axes[j].set_ylim([-0.1, 1.1])
                axes[j].grid(True, linestyle='--', alpha=0.5, axis='y')
                # set top and right spines to invisible
                axes[j].spines['top'].set_visible(False)
                axes[j].spines['right'].set_visible(False)

                if j == 0:# Add y-axis label to the first plot in each row
                    axes[j].set_ylabel("TPR@1%FPR")
                else:# disable y-axis labels for all but the first column
                    plt.setp(axes[j].get_yticklabels(), visible=False)
                    plt.setp(axes[j].get_yticklines(), visible=False)

                for model in models: # lines
                    model_df = wm_df[wm_df['model_id'] == model]
                    # Check if the model_df is empty
                    if model_df.empty:
                        print(f"Warning: No data for {attack_name}, {wm_method}, {model}\n")
                        continue

                    if attack_name == 'no_attack':
                        # No need to order the attack strengths for the no attack case
                        strengths = model_df['attack_strength'].unique()
                        results = model_df['tpr_empirical'].values
                        ci_lower = model_df['tpr_ci_lower_percentile'].values
                        ci_upper = model_df['tpr_ci_upper_percentile'].values
                    else:
                        strengths, results, ci_lower, ci_upper = order_attack_strengths(
                            ATTACK_NAME_MAPPING[attack_name]['order'],
                            model_df['attack_strength'], 
                            model_df['tpr_empirical'],
                            model_df['tpr_ci_lower_percentile'],
                            model_df['tpr_ci_upper_percentile'],
                            ATTACK_NAME_MAPPING[attack_name]['cast_to_int'],
                        )
                    
                    label = MODEL_NAME_MAPPING[model]['name']
                    
                    # Plot using actual strength values
                    line, = axes[j].plot(strengths, results,
                                marker=MODEL_NAME_MAPPING[model]['marker'],
                                linestyle=MODEL_NAME_MAPPING[model]['line'],
                                label=label,
                                color=MODEL_NAME_MAPPING[model]['color'],
                                markersize=MODEL_NAME_MAPPING[model]['size'],
                                linewidth=MODEL_NAME_MAPPING[model]['linewidth'],

                                )
                    
                    if (not np.isnan(ci_lower).any() and not np.isnan(ci_upper).any()) or (len(ci_lower) > 0 and len(ci_upper) > 0):
                        axes[j].fill_between(strengths, ci_lower, ci_upper, color=MODEL_NAME_MAPPING[model]['color'], alpha=0.2)
                        if attack_name == 'no_attack':
                            axes[j].plot(strengths, ci_lower, color=MODEL_NAME_MAPPING[model]['color'], alpha=0.2, marker='x', linestyle='--')
                            axes[j].plot(strengths, ci_upper, color=MODEL_NAME_MAPPING[model]['color'], alpha=0.2, marker='x', linestyle='--')

                                
                    if label not in labels:
                        handles.append(line)
                        labels.append(label)

                    # Set only the actual strength values as ticks
                    axes[j].set_xticks(strengths)
                    axes[j].set_xticklabels(strengths, fontsize=fs_xticks)
                    #axes[j].set_xlim([strengths[0]-0.1, strengths[-1]+0.1])
            
            

    
    fig.legend(loc='lower center', bbox_to_anchor=(0.2, -0.07, 0.5, 0.5), ncol=len(models), handles=handles, labels=labels)

    plt.savefig(args.output_plot, bbox_inches='tight', dpi=300)
    #plt.show()
    plt.close()
    print(f"Plot saved to {args.output_plot}")



In [3]:
def plot_tpr_per_metric(args, results_df, metric_name, metric_column, title_suffix, xlabel, xlim):
    """
    Generic plotting function that can use any metric for the x-axis
    
    Parameters:
    - args: The command line arguments
    - metric_name: String identifier for the metric (used in filenames)
    - metric_column: Name of the column to use for x-axis values
    - title_suffix: Text to add to the plot title
    - xlabel: Label for the x-axis
    """
    # results_df = pd.read_csv(args.output_csv)
    
    attack_names = results_df['attack_name'].unique()
    wm_methods = results_df['wm_method'].unique()
    models = results_df['model_id'].unique()

    attack_names = np.array(sorted(attack_names, key=lambda x: list(ATTACK_NAME_MAPPING.keys()).index(x)))
    wm_methods = np.array(sorted(wm_methods, key=lambda x: list(METHODS_NAME_MAPPING.keys()).index(x)))
    models = np.array(sorted(models, key=lambda x: list(MODEL_NAME_MAPPING.keys()).index(x)))

    # Setup figure with same layout
    ncols = wm_methods.shape[0] + 1 # per method, plus one for title
    nrows = attack_names.shape[0]  # for each attack
    fs = 10
    fs_xticks = 8
    fs_yticks = 8
    fs_title = 14
    y_adj = 0.937
    title_height_ratio = 0.15
    title = (
        f'Watermarking performance vs {title_suffix}\n'
        f'for dataset "{args.prompt_dataset}" for experiments in \n'
        f'{args.dataset_identifier}'
    )

    fig, gs, title_axes = setup_gridspec_figure(
        nrows=nrows, ncols=ncols,
        fs=fs, title=title, fs_title=fs_title,
        y_adj=y_adj, title_height_ratio=title_height_ratio,
        sp_width=2, sp_height=1.75
    )

    # # Set row titles (attack names)
    # for i, ax in enumerate(title_axes):
    #     ax.text(0.5, 0.25, ATTACK_NAME_MAPPING[attack_names[i]]['name'], 
    #             fontsize=fs_title, fontweight="bold", ha="center", va="center")
                      
    handles, labels = [], []
    xticks_num = 3
    xticks_stepsize = (xlim[1] - xlim[0]) / xticks_num
    xticks = np.round(np.arange(xlim[0], xlim[1] + xticks_stepsize, xticks_stepsize), 2)
    xlim_buffer = np.abs(xlim[1] - xlim[0]) * 0.07
    xlim = (xlim[0] - xlim_buffer, xlim[1] + xlim_buffer)

    # Loop through attacks and watermarking methods
    for i, attack_name in enumerate(attack_names): # rows
        attack_df = results_df[results_df['attack_name'] == attack_name]
        if attack_name not in ATTACK_NAME_MAPPING:
            continue

        axes = [fig.add_subplot(gs[2*i +1, j]) for j in range(ncols)]
        for j, wm_method in enumerate(np.concatenate((wm_methods, ["title"]))): # columns
            if wm_method == "title": # last column is title of the attack
                axes[j].axis('off')
                axes[j].text(0.1, 0.5, ATTACK_NAME_MAPPING[attack_name]['name'], fontsize=fs, fontweight="bold", ha="left", va="center")
                if i == 0:
                    axes[j].set_title('Attacktype', fontsize=fs)
            else:
                wm_df = attack_df[attack_df['wm_method'] == wm_method]

                # Set axis direction based on attack type
                if ATTACK_NAME_MAPPING[attack_name]['order'] == 'low-to-high':
                    axes[j].invert_xaxis()
                    
                if i == 0:
                    axes[j].set_title(METHODS_NAME_MAPPING[wm_method], fontsize=fs)
                
                axes[j].set_ylim([-0.1, 1.1])
                axes[j].set_yticks(np.arange(0, 1.1, 0.25))
                axes[j].set_yticklabels(np.arange(0, 1.1, 0.25), fontsize=fs_yticks)
                axes[j].set_xlim(xlim)
                axes[j].set_xticks(xticks)
                axes[j].set_xticklabels(xticks, fontsize=fs_xticks)
                axes[j].grid(True, linestyle='--', alpha=0.5)
                # set top and right spines to invisible
                axes[j].spines['top'].set_visible(False)
                axes[j].spines['right'].set_visible(False)

                if j == 0:# Add y-axis label to the first plot in each row
                    axes[j].set_ylabel("TPR@1%FPR")
                else:# disable y-axis labels for all but the first column
                    plt.setp(axes[j].get_yticklabels(), visible=False)
                    plt.setp(axes[j].get_yticklines(), visible=False)
                
                # For quality metrics (like CLIP similarity score), higher is better, 
                # so have higher values to the left
                if "score" in metric_column.lower() or "similarity" in metric_column.lower():
                    #print(f"enter score for {metric_column}")
                    if axes[j].get_xlim()[0] < axes[j].get_xlim()[1]:  # If lower values are on left
                        #print(f"enter score for {metric_column} invert")
                        axes[j].invert_xaxis()  # Invert so higher values are on left
                # For distance metrics (like FID), lower is better, so have lower values to the left
                if "fid" in metric_column.lower() or "distance" in metric_column.lower():
                    #print(f"enter fid for {metric_column}")
                    if axes[j].get_xlim()[0] > axes[j].get_xlim()[1]:  # If higher values are on left
                        #print(f"enter fid for {metric_column} invert")
                        axes[j].invert_xaxis()  # Invert so lower values are on left

                for model in models: # lines
                    model_df = wm_df[wm_df['model_id'] == model]
                    
                    # Check if the metric column exists
                    if metric_column not in model_df.columns:
                        print(f"Warning: {metric_column} not found for {attack_name}, {wm_method}, {model}")
                        continue

                    # Sort by the metric column
                    df_sorted = model_df.sort_values(by=metric_column)
                    x_values = df_sorted[metric_column].values
                    tpr_values = df_sorted['tpr_empirical'].values
                    attack_strengths = df_sorted['attack_strength'].values

                    label = MODEL_NAME_MAPPING[model]['name']
                    
                    line, = axes[j].plot(x_values, tpr_values,
                                marker=MODEL_NAME_MAPPING[model]['marker'],
                                linestyle=MODEL_NAME_MAPPING[model]['line'],
                                label=label,
                                color=MODEL_NAME_MAPPING[model]['color'])
                    
                    # Add attack strength as text near each point for reference
                    for k, (x, y, strength) in enumerate(zip(x_values, tpr_values, attack_strengths)):
                        if k % 2 == 0:  # Only label every other point to avoid clutter
                            axes[j].annotate(f"{strength}", (x, y), 
                                            textcoords="offset points", 
                                            xytext=(0, 5), 
                                            ha='center',
                                            fontsize=7)
                                
                    if label not in labels:
                        handles.append(line)
                        labels.append(label)

    
                
                

    fig.legend(loc='upper center', bbox_to_anchor=(0.2, 0.4, 0.5, 0.5), ncol=len(models), handles=handles, labels=labels)
    
    output_plot = args.output_plot.replace('.svg', f'_{metric_name}.svg')
    plt.savefig(output_plot, bbox_inches='tight', dpi=300)
    #plt.show()
    plt.close()
    print(f"\n{title_suffix} plot saved to {output_plot}")


In [11]:
args = Namespace()
args.exp_name = 'exp1'


# specify which experimental setup we want to plot
args.num_imgs = 2000
args.prompt_dataset = 'mjprompts'

# for exp1, we merge results over wmch_16 for Flux and wmch_4 for SD
args.dataset_identifier = [f'num_{args.num_imgs}_fpr_0.01_cfg_3.0_wmch_16_infsteps_50', # flux
                           f'num_{args.num_imgs}_fpr_0.01_cfg_0_wmch_16_infsteps_4', # flux_s
                           f'num_{args.num_imgs}_fpr_0.01_cfg_3.0_wmch_4_infsteps_50']  # sd

# create the output directories and ffilenames
args.input_dir = os.path.join('experiments', args.exp_name)
args.output_dir = os.path.join('experiments', args.exp_name, '_results', args.prompt_dataset,  args.dataset_identifier[0])
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

    
args.output_plot = os.path.join(args.output_dir, 'exp1_mjprompts_plot.svg')
args.merged_result_csv = os.path.join(args.output_dir, 'exp1_mjprompts_merged.csv')
# merged results already created in 5_merge_results.py
results_df = pd.read_csv(args.merged_result_csv)

#filter_attacks = np.array(['gaussian_blur_r', 'brightness_factor', 'gaussian_std'])
#filter_attacks = np.array(['crop', 'crop_scale'])
#filter_attacks = np.array(['jpeg_ratio', 'r_degree'])
filter_attacks = np.array(['adv_surr_resnet18', 'adv_embed_klvae8'])

if filter_attacks.size > 0:
    results_df = results_df[results_df['attack_name'].isin(filter_attacks)]
    args.output_plot = os.path.join(args.output_dir, f'exp1_mjprompts_plot_{filter_attacks}.svg')

# 1. plot TPR vs attack strength
plot_tpr_per_attack(args, results_df)

# # 2. plot TPR vs CLIP 
# xmin = results_df['clip_score_wm'].min()
# xmax = results_df['clip_score_wm'].max()
# plot_tpr_per_metric(
#     args, 
#     results_df, 
#     metric_name="clip_score_wm", 
#     metric_column="clip_score_wm",
#     title_suffix="CLIP similarity score",
#     xlabel="CLIP score (↑)",
#     xlim=[xmin, xmax]
# )
# xmin = results_df['clip_score_nowm'].min()
# xmax = results_df['clip_score_nowm'].max()
# plot_tpr_per_metric(
#     args, 
#     results_df, 
#     metric_name="clip_score_nowm", 
#     metric_column="clip_score_nowm",
#     title_suffix="CLIP similarity score",
#     xlabel="CLIP score (↑)",
#     xlim=[xmin, xmax]
# )

# # 3. plot TPR vs diff 
# xmin = results_df['wm_diff'].min()
# xmax = results_df['wm_diff'].max()
# plot_tpr_per_metric(
#     args, 
#     results_df, 
#     metric_name="wm_diff", 
#     metric_column="wm_diff",
#     title_suffix="Abs. Mean Difference (originial - recovered)",
#     xlabel="Diff (↓)",
#     xlim=[xmin, xmax]
# )

# # 4. plot TPR vs FID (WM vs COCO)
# xmin = results_df['fid_wm_coco'].min()
# xmax = results_df['fid_wm_coco'].max()
# plot_tpr_per_metric(
#     args, 
#     results_df, 
#     metric_name="fid_wm_coco", 
#     metric_column="fid_wm_coco",
#     title_suffix="FID (WM vs COCO)",
#     xlabel="FID (↓)",
#     xlim=[xmin, xmax]
# )

# # 5. plot TPR vs FID (WM vs NOWM)
# xmin = results_df['fid_wm_nowm'].min()
# xmax = results_df['fid_wm_nowm'].max()
# plot_tpr_per_metric(
#     args, 
#     results_df, 
#     metric_name="fid_wm_nowm", 
#     metric_column="fid_wm_nowm",
#     title_suffix="FID (WM vs NOWM)",
#     xlabel="FID (↓)",
#     xlim=[xmin, xmax]
# )

Plot saved to experiments/exp1/_results/mjprompts/num_2000_fpr_0.01_cfg_3.0_wmch_16_infsteps_50/exp1_mjprompts_plot_['adv_surr_resnet18' 'adv_embed_klvae8'].svg


# Numerical Table Results
### Calculate Mean over Attack Strengths

In [8]:
model_ids = results_df['model_id'].unique()
wm_methods = results_df['wm_method'].unique()
attack_types = results_df['attack_type'].unique()
attack_names = results_df['attack_name'].unique()
#wm_chs = results_df['wm_ch'].unique()
attack_strengths = results_df['attack_strength'].unique()

print(f"model_ids: {model_ids}")
print(f"wm_methods: {wm_methods}")
print(f"attack_types: {attack_types}")
print(f"attack_names: {attack_names}")
#print(f"wm_chs: {wm_chs}")
print(f"attack_strengths: {attack_strengths}")

def print_metric_per_attack(results_df, model_ids, wm_methods, attack_names, metric="tpr_empirical"):
    
    # loop over model_ids
    for model_id in model_ids:
        model_df = results_df[results_df['model_id'] == model_id]
        # check if empty
        if model_df.empty:
            print(f"Model ID {model_id} has no data.")
            continue
        else:
            print(f"\nModel: {model_id}")
        
        # loop over wm_methods
        for wm_method in wm_methods:
            wm_df = model_df[model_df['wm_method'] == wm_method]
            # check if empty
            if wm_df.empty:
                print(f"\tWM method {wm_method} has no data for model ID {model_id}.")
                continue
            else:
                print(f"\tWM method: {wm_method}")
            # loop over attack_types
            for attack_name in attack_names:
                attack_df = wm_df[model_df['attack_name'] == attack_name]
                # check if empty
                if attack_df.empty:
                    print(f"\t\tAttack name {attack_name} has no data for model ID {model_id} and WM method {wm_method}.")
                    continue
                else:
                    print(f"\t\tAttack name: {attack_name}")
                #for wm_ch in wm_chs:
                    #wm_ch_df = attack_df[attack_df['wm_ch'] == wm_ch]
                    # check if empty
                    #if wm_ch_df.empty:
                        #print(f"\n\t\t\tWM channel {wm_ch} has no data for attack name {attack_name}, model ID {model_id}, and WM method {wm_method}.")
                        #continue
                    #else:
                        #print(f"\n\t\t\tWM channel: {wm_ch}")
                    
                attack_strengths = attack_df['attack_strength'].unique()
                #print(f"\t\t\t\tattack strengths: {attack_strengths}")
                #print(f"\t\t\t\ttpr empirical values: {attack_df['tpr_empirical'].values}")
                #print(attack_df)
                mean_metric = attack_df[metric].mean()
                std_metric = attack_df[metric].std()
                print(f"\t\t\t\t{metric}: {mean_metric:.3f}+-{std_metric:.3f}")
            


model_ids: ['flux_s' 'sd' 'flux']
wm_methods: ['tr' 'prc' 'grids' 'rid' 'gs']
attack_types: ['distortion' 'adversarial_surr' 'adversarial_embed' 'no_attack']
attack_names: ['gaussian_blur_r' 'crop_scale' 'adv_surr_resnet18' 'adv_embed_klvae8'
 'gaussian_std' 'brightness_factor' 'jpeg_ratio' 'crop' 'r_degree'
 'no_attack']
attack_strengths: ['1.0' '2.0' '3.0' '4.0' '5.0' '0.9' '0.8' '0.7' '0.6' '0.5' '0.0' '10.0'
 '0.02' '0.04' '0.06' '0.08' '0.1' '1.2' '1.4' '1.6' '1.8' '90.0' '70.0'
 '50.0' '30.0' 'no_attack']


In [10]:
#TPR
print_metric_per_attack(results_df, model_ids, wm_methods, attack_names, metric="tpr_empirical")


Model: flux_s
	WM method: tr
		Attack name: gaussian_blur_r
				tpr_empirical: 0.048+-0.042
		Attack name: crop_scale
				tpr_empirical: 0.074+-0.110
		Attack name: adv_surr_resnet18
				tpr_empirical: 0.489+-0.047
		Attack name: adv_embed_klvae8
				tpr_empirical: 0.418+-0.145
		Attack name: gaussian_std
				tpr_empirical: 0.459+-0.117
		Attack name: brightness_factor
				tpr_empirical: 0.161+-0.073
		Attack name: jpeg_ratio
				tpr_empirical: 0.086+-0.071
		Attack name: crop
				tpr_empirical: 0.008+-0.002
		Attack name: r_degree
				tpr_empirical: 0.011+-0.003
		Attack name: no_attack
				tpr_empirical: 0.540+-nan
	WM method: prc
		Attack name: gaussian_blur_r
				tpr_empirical: 0.136+-0.272
		Attack name: crop_scale
				tpr_empirical: 0.106+-0.171
		Attack name: adv_surr_resnet18
				tpr_empirical: 0.478+-0.302
		Attack name: adv_embed_klvae8
				tpr_empirical: 0.462+-0.297
		Attack name: gaussian_std
				tpr_empirical: 0.199+-0.150
		Attack name: brightness_factor
				tpr_empiric

/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index

In [11]:
#CLIP scores
print_metric_per_attack(results_df, model_ids, wm_methods, attack_names, metric="clip_score_wm")


Model: flux_s
	WM method: tr
		Attack name: gaussian_blur_r
				clip_score_wm: 0.323+-0.002
		Attack name: crop_scale
				clip_score_wm: 0.323+-0.004
		Attack name: adv_surr_resnet18
				clip_score_wm: 0.327+-0.001
		Attack name: adv_embed_klvae8
				clip_score_wm: 0.327+-0.001
		Attack name: gaussian_std
				clip_score_wm: 0.325+-0.000
		Attack name: brightness_factor
				clip_score_wm: 0.325+-0.001
		Attack name: jpeg_ratio
				clip_score_wm: 0.323+-0.003
		Attack name: crop
				clip_score_wm: 0.324+-0.003
		Attack name: r_degree
				clip_score_wm: 0.328+-0.002
		Attack name: no_attack
				clip_score_wm: 0.328+-nan
	WM method: prc
		Attack name: gaussian_blur_r
				clip_score_wm: 0.324+-0.002
		Attack name: crop_scale
				clip_score_wm: 0.324+-0.004
		Attack name: adv_surr_resnet18
				clip_score_wm: 0.329+-0.000
		Attack name: adv_embed_klvae8
				clip_score_wm: 0.327+-0.001
		Attack name: gaussian_std
				clip_score_wm: 0.325+-0.000
		Attack name: brightness_factor
				clip_score_

/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index

In [9]:
#FID
print_metric_per_attack(results_df, model_ids, wm_methods, attack_names, metric="fid_wm_coco")


Model: flux_s
	WM method: tr
		Attack name: gaussian_blur_r
				fid_wm_coco: 102.555+-9.559
		Attack name: crop_scale
				fid_wm_coco: 86.889+-0.406
		Attack name: adv_surr_resnet18
				fid_wm_coco: 85.609+-0.370
		Attack name: adv_embed_klvae8
				fid_wm_coco: 87.036+-2.893
		Attack name: gaussian_std
				fid_wm_coco: 87.994+-2.619
		Attack name: brightness_factor
				fid_wm_coco: 88.347+-1.360
		Attack name: jpeg_ratio
				fid_wm_coco: 89.097+-4.743
		Attack name: crop
				fid_wm_coco: 87.526+-0.776
		Attack name: r_degree
				fid_wm_coco: 89.352+-1.345
		Attack name: no_attack
				fid_wm_coco: 86.201+-nan
	WM method: prc
		Attack name: gaussian_blur_r
				fid_wm_coco: 103.303+-10.112
		Attack name: crop_scale
				fid_wm_coco: 86.798+-0.574
		Attack name: adv_surr_resnet18
				fid_wm_coco: 85.572+-0.530
		Attack name: adv_embed_klvae8
				fid_wm_coco: 86.602+-2.715
		Attack name: gaussian_std
				fid_wm_coco: 87.142+-2.518
		Attack name: brightness_factor
				fid_wm_coco: 86.987+-0.

/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  attack_df = wm_df[model_df['attack_name'] == attack_name]
/tmp/ipykernel_1404462/1342516184.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index